# Large MNL work

Sam Maurer, April 2018 (last updated June 2018) - Python 3.6

This notebook is for development, testing, and demonstration of the template for MNL with large numbers of alternatives.

In [1]:
import os; os.chdir('../')

In [2]:
import numpy as np
import pandas as pd

In [3]:
from urbansim_templates import modelmanager as mm
from urbansim_templates.models import LargeMultinomialLogitStep
import orca

/Users/maurer/anaconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [4]:
# Load any script-based Orca registrations
from scripts import datasources
from scripts import models

In [5]:
for table_name in orca.list_tables():
    print(table_name.upper())
    print(orca.get_table(table_name).to_frame().columns.tolist())
    print()

PARCELS
['development_type_id', 'land_value', 'acres', 'county_id', 'zone_id', 'proportion_undevelopable', 'tax_exempt_status', 'apn', 'parcel_id_local', 'geom_id', 'imputation_flag', 'x', 'y', 'shape_area', 'block_id', 'node_id']

BUILDINGS
['parcel_id', 'development_type_id', 'improvement_value', 'residential_units', 'residential_sqft', 'sqft_per_unit', 'non_residential_sqft', 'building_sqft', 'nonres_rent_per_sqft', 'res_price_per_sqft', 'stories', 'year_built', 'redfin_sale_price', 'redfin_sale_year', 'redfin_home_type', 'costar_property_type', 'costar_rent', 'building_type_id']

UNITS
['Unnamed: 0', 'building_id', 'num_units', 'tenure', 'unit_num', 'unit_residential_price', 'unit_residential_rent']

HOUSEHOLDS
['household_id', 'serialno', 'persons', 'building_type', 'cars', 'income', 'race_of_head', 'hispanic_head', 'age_of_head', 'workers', 'state', 'county', 'tract', 'block group', 'children', 'tenure', 'recent_mover', 'block_group_id', 'single_family', 'unit_id']

PERSONS
['Unn

# Large MNL

In [6]:
m = LargeMultinomialLogitStep()
m.choosers = ['households']
m.alternatives = ['units', 'buildings']
m.choice_column = 'unit_id'
m.alt_sample_size = 10
m.chooser_filters = ['household_id % 1000 < 1']

m.model_expression = 'res_price_per_sqft - 1'

m.name = 'large-mnl-test'
m.tags = ['sam', 'testing']

In [7]:
len(m._get_df(tables=m.choosers, filters=m.chooser_filters))

2680

In [8]:
%%time
m.fit()

                  CHOICEMODELS ESTIMATION RESULTS                  
Dep. Var.:                chosen   No. Observations:               
Model:         Multinomial Logit   Df Residuals:                   
Method:       Maximum Likelihood   Df Model:                       
Date:                              Pseudo R-squ.:                  
Time:                              Pseudo R-bar-squ.:              
AIC:                               Log-Likelihood:       -5,717.319
BIC:                               LL-Null:              -5,717.319
                        coef   std err         z     P>|z|   Conf. Int.
-----------------------------------------------------------------------
res_price_per_sqft   -0.0000     0.000    -0.009                       
CPU times: user 2.7 s, sys: 1.62 s, total: 4.33 s
Wall time: 4.3 s


In [9]:
m.fitted_parameters

[-2.941252961766151e-07]

In [10]:
m.register()

In [11]:
mm.get_step('large-mnl-test').fitted_parameters

[-2.941252961766151e-07]

In [12]:
m.out_chooser_filters = ['household_id % 1000 == 17']

In [13]:
df = orca.get_table('units').to_frame()
print(df.index)
df.index.name = 'unit_id'
print(df.index)

RangeIndex(start=0, stop=2784008, step=1, name='unit_id')
RangeIndex(start=0, stop=2784008, step=1, name='unit_id')


In [14]:
%%time
m.run()

CPU times: user 2.53 s, sys: 1.56 s, total: 4.09 s
Wall time: 4.06 s


In [16]:
print(m.probabilities.head())

   observation_id  unit_id  probability
0              17  1659707     0.100000
1              17   917898     0.100003
2              17   823956     0.099994
3              17   501589     0.100010
4              17  1905583     0.100013


In [17]:
print(m.choices.head())

17      1659707
1017    1107809
2017     652905
3017    1557673
4017    1102333
Name: choice, dtype: int64
